# Database (pre)Pre-processing
<hr>

The original data, when pulled from NYC's OpenData, is incompatible with pandas sql uploader. So in order to store it to our MySQL server, we need to diagnose the issue, fix it, and get the data up so it's easily accessible.

In [52]:
import pandas as pd

df = pd.read_csv('crimeStat.csv')

C:\Users\Ambrose\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3,5,10,11,15,16,18,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### when first attempting the upload, it seemed like the biggest hangup was "lon_lat" which is redundent info anyway, so well make sure that longitude and lon_lat have the same amount of NaNs and drop it if they're equal

In [53]:
# resetting the index because it cycles through 1-999

df = df.reset_index(drop=True)

In [54]:
df['longitude'].isnull().sum()

6

In [55]:
df['lon_lat'].isnull().sum()

76

In [56]:
# the nulls are different, so lets see the difference

longMask = (df['longitude'].isnull())
lonLatMask = (df['lon_lat'].isnull())

In [57]:
df[longMask]

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,...,x_coord_cd,y_coord_cd,latitude,longitude,lon_lat,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
0,32311380,2007-06-18T00:00:00.000,511,"CONTROLLED SUBSTANCE, POSSESSION 7",235,DANGEROUS DRUGS,PL 2200300,M,Q,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234367,71025027,2010-01-29T00:00:00.000,475,PL 1651600,M,B,42,1,25-44,M,...,-73.8808,"{'type': 'Point', 'coordinates': [-73.88075096...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3094051,78871391,2011-05-20T00:00:00.000,759,PL 1950500,M,M,33,0,25-44,F,...,-73.9283,"{'type': 'Point', 'coordinates': [-73.92825431...",NaN,NaN,NaN,NaN,NaN,"PUBLIC ADMINISTATION,UNCLASS MISDEMEAN 4",359.0,OFFENSES AGAINST PUBLIC ADMINISTRATION
3094361,78315429,2011-04-19T00:00:00.000,567,PL 2211001,M,B,52,0,25-44,M,...,-73.8788,"{'type': 'Point', 'coordinates': [-73.87879308...",NaN,NaN,NaN,NaN,NaN,"MARIJUANA, POSSESSION 4 & 5",235.0,DANGEROUS DRUGS
3094443,78564530,2011-05-04T00:00:00.000,109,PL 1200501,F,Q,114,0,25-44,M,...,-73.8954,"{'type': 'Point', 'coordinates': [-73.89543217...",NaN,NaN,NaN,NaN,NaN,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT
3094451,78852532,2011-05-19T00:00:00.000,905,VTL11920M2,M,Q,109,0,25-44,M,...,-73.7698,"{'type': 'Point', 'coordinates': [-73.76981885...",NaN,NaN,NaN,NaN,NaN,"INTOXICATED DRIVING,ALCOHOL",347.0,INTOXICATED & IMPAIRED DRIVING


In [58]:
df[lonLatMask]

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,...,x_coord_cd,y_coord_cd,latitude,longitude,lon_lat,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
0,32311380,2007-06-18T00:00:00.000,511,"CONTROLLED SUBSTANCE, POSSESSION 7",235,DANGEROUS DRUGS,PL 2200300,M,Q,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183032,206128002,2019-12-06T00:00:00.000,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN",348,VEHICLE AND TRAFFIC LAWS,VTL0512000,K,70,0,...,177708,40.6544,-73.963014,"{'type': 'Point', 'coordinates': [-73.96301369...",NaN,13.0,2.0,27,50.0,M
183679,204346699,2019-10-29T00:00:00.000,439,"LARCENY,GRAND FROM OPEN AREAS, UNATTENDED",109,GRAND LARCENY,PL 1553001,M,22,0,...,224726,40.7835,-73.964379,"{'type': 'Point', 'coordinates': [-73.96437917...",NaN,19.0,4.0,19,13.0,F
213441,206495842,2019-12-15T00:00:00.000,905,"INTOXICATED DRIVING,ALCOHOL",347,INTOXICATED & IMPAIRED DRIVING,VTL11920U3,Q,109,4,...,229962,40.7977,-73.826981,"{'type': 'Point', 'coordinates': [-73.82698053...",NaN,NaN,NaN,NaN,NaN,M
234367,71025027,2010-01-29T00:00:00.000,475,PL 1651600,M,B,42,1,25-44,M,...,-73.8808,"{'type': 'Point', 'coordinates': [-73.88075096...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820755,65182068,2009-08-27T00:00:00.000,511,"CONTROLLED SUBSTANCE, POSSESSION 7",235,DANGEROUS DRUGS,PL 2200300,K,75,0,...,2181178,46.1432,-73.868996,"{'type': 'Point', 'coordinates': [-73.86899582...",NaN,NaN,NaN,NaN,NaN,M
4360399,46223422,2008-05-06T00:00:00.000,567,"MARIJUANA, POSSESSION 4 & 5",235,DANGEROUS DRUGS,PL 2211001,M,22,0,...,226907,40.7895,-73.957395,"{'type': 'Point', 'coordinates': [-73.95739533...",NaN,19.0,4.0,19,13.0,M
4360549,45296703,2008-04-07T00:00:00.000,101,ASSAULT 3,344,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,22,0,...,224278,40.7823,-73.969457,"{'type': 'Point', 'coordinates': [-73.96945677...",NaN,19.0,4.0,19,13.0,M
4360860,45640672,2008-04-14T00:00:00.000,109,"ASSAULT 2,1,UNCLASSIFIED",106,FELONY ASSAULT,PL 1200501,K,72,0,...,176142,40.6502,-74.005076,"{'type': 'Point', 'coordinates': [-74.00507553...",NaN,9.0,2.0,7,45.0,F


In [59]:
# make lists out of each null values

longlist = list(df[longMask].index)
latlonlist = list(df[lonLatMask].index)

In [60]:
any(item in latlonlist for item in longlist)

True

In [61]:
all(item in latlonlist for item in longlist)

True

In [62]:
# all the nulls match, which means deleting the row will not cause any data loss

del df['lon_lat']

In [64]:
# now that that col is deleted we can try to upload it

from sqlalchemy import create_engine
from pandas.io import sql

# Database creds, don't share on public repo
hostname='95.217.156.58'
dbname='crimeStats'
uname='sandwich'
pwd='321#@!IdiotSandwich'
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

df.to_sql('crimeTableMain', engine, if_exists='replace', index=True)